# Reading the DREQ spreadsheet
This is just an attempt to see if I can do this.  This is using [openPyXL](https://openpyxl.readthedocs.io/en/default/index.html).

In [1]:
from os import makedirs
from os.path import expanduser, join, isdir
from re import split
from json import dump
from openpyxl import load_workbook

# Where things are
#
root = expanduser("~/work/cmip6-data-request")
dreq_xlsx = join(root, "CMIP6dreq/trunk/dreqPy/docs/CMIP6_MIP_tables.xlsx")
outdir = join(root, "scratch/xlout")

# Load the dreq spreadsheet
#
dreq = load_workbook(filename=dreq_xlsx,
                     read_only=True)

In [2]:
def tuplify_mips(mips):
    # turn a stringy list of mips into a tuple
    if mips is not None:
        return tuple(split(r",\s*", mips))
    else:
        return ()

In [3]:
# Find the headers of a 'good' miptable
#
amon = dreq['Amon']
headers = tuple((i, c.value) for (i, c) in enumerate(tuple(amon.rows)[0]))
headers

((0, u'Priority'),
 (1, u'Long name'),
 (2, u'units'),
 (3, u'description'),
 (4, u'comment'),
 (5, u'Variable Name'),
 (6, u'CF Standard Name'),
 (7, u'cell_methods'),
 (8, u'positive'),
 (9, u'type'),
 (10, u'dimensions'),
 (11, u'CMOR Name'),
 (12, u'modeling_realm'),
 (13, u'frequency'),
 (14, u'cell_measures'),
 (15, u'prov'),
 (16, u'provNote'),
 (17, u'rowIndex'),
 (18, u'UID'),
 (19, u'vid'),
 (20, u'stid'),
 (21, u'Structure Title'),
 (22, u'MIPs (requesting)'),
 (23, u'MIPs (by experiment)'))

In [4]:
# Build a big table mapping miptable to tuples of rows
#

# Skip sheets which are not actually miptables
#
non_miptables={'Notes'}

mtrows = {mt: tuple({'label/v': r[5].value,
                     'label/c': r[11].value,
                     'mips/r': tuplify_mips(r[22].value),
                     'mips/e': tuplify_mips(r[23].value)}
                    for r in tuple(dreq[mt].rows)[1:])
          for mt in dreq.sheetnames
          if mt not in non_miptables}

In [5]:
# And finally spit the whole thing out
#
if not isdir(outdir):
    makedirs(join(outdir))

with open(join(root, outdir, "ts.json"), 'w') as out:
    dump(mtrows, out, indent=2, sort_keys=True)